<a href="https://colab.research.google.com/github/UtkarshRasal/MusicSearchAI/blob/master/MusicSeachExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir data

In [ ]:
!pip install transformers sentence-transformers faiss-cpu pandas

In [5]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
df = pd.read_csv("/content/data/songs.csv")

df

,title,artist,lyrics
0,Someone Like You,Adele,I heard that youre settled down that you found...
1,Let Her Go,Passenger,Well you only need the light when its burning ...
2,Perfect,Ed Sheeran,I found a love for me darling just dive right ...
3,Shape of You,Ed Sheeran,The club isnt the best place to find a lover s...
4,Blinding Lights,The Weeknd,Ive been tryna call Ive been on my own for lon...
5,Fix You,Coldplay,When you try your best but you dont succeed wh...
6,Photograph,Ed Sheeran,We keep this love in a photograph we made thes...
7,Stay,Rihanna,All along it was a fever a cold sweat hotheade...
8,Counting Stars,OneRepublic,Lately I been I been losing sleep dreaming abo...
9,Love Yourself,Justin Bieber,For all the times that you rain on my parade a...


In [7]:
# Combine title + lyrics for embedding
texts = (df['title'] + '-' + df['lyrics']).tolist()
texts

['Someone Like You-I heard that youre settled down that you found a girl and youre married now',
 'Let Her Go-Well you only need the light when its burning low only miss the sun when it starts to snow',
 'Perfect-I found a love for me darling just dive right in and follow my lead',
 'Shape of You-The club isnt the best place to find a lover so the bar is where I go',
 'Blinding Lights-Ive been tryna call Ive been on my own for long enough maybe you can show me how to love',
 'Fix You-When you try your best but you dont succeed when you get what you want but not what you need',
 'Photograph-We keep this love in a photograph we made these memories for ourselves',
 'Stay-All along it was a fever a cold sweat hotheaded believer',
 'Counting Stars-Lately I been I been losing sleep dreaming about the things that we could be',
 'Love Yourself-For all the times that you rain on my parade and all the clubs you get in using my name']

In [13]:
# Create embeddings
embeddings = model.encode(texts, show_progress_bar=True)

# Save embeddings with FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype("float32"))

# Save index and metadata
# create a folder "faiss" if some faiss error occurs
faiss.write_index(index, "faiss/song_index.index")
df.to_csv("faiss/song_metadata.csv", index=False)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
!mkdir -p faiss

In [14]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
index = faiss.read_index("faiss/song_index.index")
df = pd.read_csv("faiss/song_metadata.csv")

def search_similar_songs(query: str, top_k: int = 5):
    query_embedding = model.encode([query])
    query_embedding = np.array(query_embedding).astype("float32")

    D, I = index.search(query_embedding, top_k)

    print(f"\n🔍 Top {top_k} Results for: '{query}'\n")
    for rank, idx in enumerate(I[0]):
        row = df.iloc[idx]
        print(f"{rank+1}. {row['title']} – {row['artist']}")
        print(f"    Lyrics: {row['lyrics'][:100]}...\n")


In [17]:
search_similar_songs('song about cold sweat')


🔍 Top 5 Results for: 'song about cold sweat'

1. Stay – Rihanna
    Lyrics: All along it was a fever a cold sweat hotheaded believer...

2. Blinding Lights – The Weeknd
    Lyrics: Ive been tryna call Ive been on my own for long enough maybe you can show me how to love...

3. Love Yourself – Justin Bieber
    Lyrics: For all the times that you rain on my parade and all the clubs you get in using my name...

4. Shape of You – Ed Sheeran
    Lyrics: The club isnt the best place to find a lover so the bar is where I go...

5. Perfect – Ed Sheeran
    Lyrics: I found a love for me darling just dive right in and follow my lead...

